In [1]:
import os
import time
from pathlib import Path
import sys
import pickle
import joblib

#Import data manipulation libaries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#import ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Splitting the data and cross-validation
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score,KFold, GroupKFold, StratifiedKFold

#Metrics
#from sklearn.metrics import classification_report,accuracy_score , roc_auc_score
from sklearn.metrics import mean_squared_error, roc_auc_score, log_loss, mean_squared_log_error
#Classifiers
import xgboost as xgb
from lightgbm import LGBMRegressor, LGBMClassifier

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

#Feature engineerring
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_transformer

#Encoding

import category_encoders as ce

In [3]:
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import SVR

from xgboost import XGBRegressor
from xgboost import XGBClassifier

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

In [4]:
#New
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import gc

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from mlxtend.preprocessing import minmax_scaling
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from scipy import stats
import optuna
import xgboost as xgb
from sklearn.ensemble import (RandomForestClassifier, RandomForestRegressor,AdaBoostClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, VotingClassifier,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.utils import class_weight
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.svm import SVR
from sklearn import datasets, linear_model
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.signal import argrelmin
from scipy.stats import skew
from scipy import stats


pd.set_option('display.max_columns', None)   

from sklearn.model_selection import train_test_split

import eli5
from eli5.sklearn import PermutationImportance
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from tensorflow.keras import utils
import tensorflow_addons as tfa
import keras_tuner
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters, Objective

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:78: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.


In [ ]:
train_import = pd.read_csv("/kaggle/input/playground-series-s3e11/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s3e11/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s3e11/sample_submission.csv")

In [ ]:
train_import.info()

In [ ]:
train_import.head(5)

In [ ]:
def create_EDA_summary (df = None):
    train_import_table = pd.DataFrame()
    train_import_table["dtype"] = train_import.apply(lambda x: x.dtype)
    train_import_table["NROW"] = train_import.shape[0]
    train_import_table["Unique_values"] = [ len(train_import[col].unique()) for col in train_import.columns]
    train_import_table["Percent_missing"] = (train_import.isnull().sum()/train_import.shape[0])*100
    
    return train_import_table

In [ ]:
#train_import = train_import.copy()
create_EDA_summary (df = train_import)

#### Creating the training labels and training features

In [ ]:
train_import = train_import.sample(frac = 1)

In [ ]:
#Create labels and starting feature set

# Define the target
target = "cost"

# Define the training labels
train_y = train_import[target]
#Define the starting training features
train_df_start = train_import.drop(target , axis = 1)

#del train_import

In [ ]:
train_df_start.shape, train_y.shape , type(train_y)

#### Create the different list of data types for feature engineering

#### Define the cross-validation strategy

In [ ]:
splitter = KFold(n_splits=5, shuffle=True, random_state=42)

## Feature engineering

1.) Cast features to correct data type

In [ ]:
#Initial casting of features to reduce mem_size
int_features = [ col for col in train_df_start.columns  if train_df_start[col].nunique() <50]

for col in int_features: 
    train_df_start[col] = train_df_start[col].astype('int16')

In [ ]:
train_df_start.head(2)

In [ ]:
#Create new features
#Empty at the moment
train_df = train_df_start.copy()

dataset = [train_df, test_df]

features_facilities = ['coffee_bar', 'video_store', 'salad_bar', 'prepared_food', 'florist']

for df in dataset:
    df['sales_per_unit'] = df['store_sales(in millions)'] / df['unit_sales(in millions)']
    df['sales_per_sqft'] = df['store_sales(in millions)'] / df['store_sqft']
#    df['cars_per_households'] = df['avg_cars_at home(approx).1'] / (df['num_children_at_home'] + 2)
    df['has_all_facilities'] = df[features_facilities].all(axis=1)
    df['facilities_rate'] = df[features_facilities].sum(axis=1) / len(features_facilities)




del train_df_start

In [ ]:
test_df

In [ ]:
#Delete specific features

drop_features = []
#drop_features = ['low_fat', 'gross_weight', 'recyclable_package', 'units_per_case', 'unit_sales(in millions)', "id"]

In [ ]:
# List of features
feature_list = [col for col in train_df.columns if col not in drop_features]

predictors = feature_list



In [ ]:
feature_list

In [ ]:
for col in int_features: 
    train_df[col] = train_df[col].astype('int64')


In [ ]:
train_df.info()

#### Make Feature engineering pipeline

In [ ]:
#Define the fit_transformation steps
#pre_num = Pipeline(steps=[('scaler', StandardScaler())])

#preprocess_binary = Pipeline(steps=[('label_encoder',
#ce.binary.BinaryEncoder(return_df=True))])

#preprocess_high_cardinality = Pipeline(steps=[('high_cardinal',
#ce.TargetEncoder(cols = high_cardinal))])

In [ ]:
# #ct = ColumnTransformer(
# transformers=[('num', #A
# pre_num, num_features)],
# remainder='passthrough')

#### Define the model

In [ ]:
#model = LGBMRegressor(objective = 'regression', metric = 'reg:squaredlogerror', device_type = "cpu")
#model_lgbm_v1 = LGBMRegressor(objective = 'regression', metric = "rmse", device_type = "gpu")

### Define the final list of features and the target for the test and training data set

In [ ]:
train = train_df.sample(frac = 1).copy().reset_index()
train_X = train.loc[:, predictors]
train_y = train_y
test_X = test_df.loc[:, predictors]
train_X.shape, train_y.shape, test_X.shape

del train

#### Define the training loop for cross-validation

In [ ]:
train_score_opt = []
valid_score_opt = []

for i in [ 0.005, 0.015, 0.025, 0.05, 0.1]:
    
    train_list = [] # An numpy array with a length of the number of folds
    val_list = []

    #for i, rate in enumerate(learning_rates): #D
    for idx , (trn, val) in enumerate(splitter.split(train_X)): #Create indexes for the train and validation set  

        Xtrain = train_X.iloc[trn]
        Xval = train_X.iloc[val]
        ytrain = train_y.iloc[trn].values.ravel()
        yval = train_y.iloc[val].values.ravel()

        #Xtrain_prepocessed = ct.fit_transform(Xtrain, ytrain)
        #Xval_preprocessed = ct.transform(Xval)

        model  = model = LGBMRegressor(objective = 'regression', metric = 'reg:squaredlogerror', device_type = "cpu", learning_rate = i)
        model.fit(Xtrain, ytrain)

        train_pred =  model.predict(Xtrain)
        train_score = mean_squared_log_error(ytrain, train_pred, squared=False)


        valid_pred = model.predict(Xval)
        valid_score =  mean_squared_log_error(yval, valid_pred,squared=False )

        train_list.append(train_score)
        val_list.append(valid_score)
        # Append the results from the importance score to a list
        #print(f"The average training RMSE_log is {np.mean(train_list):.4f}")
        #print(f"The average validation RMSE_log is {np.mean(val_list):.4f}")
    
    train_score_opt.append(train_list)
    valid_score_opt.append(val_list)
    print(f"The average training RMSE_log for the learning rate {i} is {np.mean(train_score_opt)}")
    print(f"The average validation RMSE_log for the learning rate {i} is {np.mean(valid_score_opt)}")

In [ ]:
lr_x = [ 0.005, 0.015, 0.025, 0.05, 0.1]
lr_rate = [np.mean(valid_score_opt[i]) for i in range(len(valid_score_opt))]

In [ ]:
fig, ax = plt.subplots()
ax.plot(lr_x, lr_rate)
plt.show()

#### Optuna

#### Define the experiment tracking strategy

#### Step 1: Describe the experiment and what will be tested and why

In [ ]:
descrip_exp_1 = "Do not drop the binary features and test the result"

In [ ]:
data = {'experiment1': [descrip_exp_1, best_params, best_value, best_trial], 'col_2': [descrip_exp_1, best_params, best_value, best_trial]}
data = pd.DataFrame.from_dict(data )

In [ ]:
data.index = ["Experiment", "Best Model", "test", "rr"]

In [ ]:
database = data
database

In [ ]:
data_new = {'experiment3': [descrip_exp_1, best_params, best_value, best_trial]}
data_new = pd.DataFrame.from_dict(data_new)
data_new

In [ ]:
data_new.index = ["Experiment", "Best Model", "Best value", "rr"]

Step: 1 --- Load dataframe into memory
Step: 2 --- 